In [ ]:
import pandas as pd

submission = pd.read_csv('../input/subsss/submission2.csv')
submission.to_csv("submission.csv", index=False)
submission

In [ ]:
#!pip install detectron2 -f \
  #https://dl.fbaipublicfiles.com/detectron2/wheels/cpu/torch1.7/index.html

In [ ]:
import torch
#from detectron2 import model_zoo
#from detectron2.config import get_cfg, CfgNode
#from detectron2.data import DatasetCatalog, MetadataCatalog,build_detection_train_loader, build_detection_test_loader, transforms, DatasetMapper, detection_utils
#from detectron2.data.datasets import load_coco_json
#from detectron2.engine import DefaultPredictor,DefaultTrainer
#from detectron2.evaluation import COCOEvaluator, inference_on_dataset
#from detectron2.structures import BoxMode
#from detectron2.utils.logger import setup_logger
#setup_logger()
#from detectron2.utils.visualizer import ColorMode, Visualizer
from tqdm import tqdm

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
import cv2
import random

import warnings
warnings.filterwarnings('ignore')
import os
import shutil as sh
import yaml
import re

from sklearn.utils import shuffle
import albumentations as A
from copy import deepcopy

In [ ]:
df=pd.read_csv('../input/vinbigdata-testmeta/test_meta.csv')
df.rename(columns={'dim0' : 'height','dim1':'width'},inplace=True)
df.head()

In [ ]:
Data_Dir="../input/vinbigdata-chest-xray-abnormalities-detection"

Train_Dir=os.path.join(Data_Dir,'train')
Test_Dir=os.path.join(Data_Dir,'test')

Train_Dir_Files=[os.path.join(Train_Dir,name) for name in os.listdir(Train_Dir)]
Test_Dir_Files=[os.path.join(Test_Dir,name) for name in os.listdir(Test_Dir)]

print("The total number of train dicom files :",len(Train_Dir_Files))
print("The total number of test dicom files :",len(Test_Dir_Files))

In [ ]:
Data_Dir_Resized='../input/vinbigdata-chest-xray-resized-png-256x256'

Train_Dir_Resized=f"{Data_Dir_Resized}/train"
Test_Dir_Resized=f"{Data_Dir_Resized}/test"

Train_Dir_Files_Resized=[os.path.join(Train_Dir_Resized,name) for name in os.listdir(Train_Dir_Resized)]
Test_Dir_Files_Resized=[os.path.join(Test_Dir_Resized,name) for name in os.listdir(Test_Dir_Resized)]

print("The total number of train dicom files :",len(Train_Dir_Files))
print("The total number of test dicom files :",len(Test_Dir_Files))

In [ ]:
train_csv=pd.read_csv(f"{Data_Dir}/train.csv")
print("Training DataFrame :",format(len(train_csv)),"entries")
train_csv.head()

In [ ]:
train_resized=pd.read_csv(f"{Data_Dir_Resized}/train_meta.csv")
train_resized.rename(columns={'dim0':'height','dim1':'width'},inplace=True)
print("Metadata for Resized Training Images :",format(len(train_resized)),"entries")
train_resized.head()

In [ ]:
test_resized=pd.read_csv('../input/vinbigdata-testmeta/test_meta.csv')
test_resized.rename(columns={'dim0':'height','dim1':'width'},inplace=True)
print("Metadata for Resized Test Images :",format(len(test_resized)),"entries")
test_resized.head()

In [ ]:
sub_csv=pd.read_csv("../input/vinbigdata-chest-xray-abnormalities-detection/sample_submission.csv")
print("Sample Submission DataFrame")
sub_csv.head()

In [ ]:
image_ids=pd.DataFrame(train_csv['image_id'].unique(),columns=['Image_ids'])
print("Unique Image Ids :",len(image_ids),"entries")
image_ids.head()

In [ ]:
classes=pd.DataFrame(train_csv['class_name'].unique(),columns=['class_name'])
classes['class_id']=train_csv['class_id'].unique()
classes.set_index(['class_id'],inplace=True)
classes.sort_values(['class_id'],inplace=True)
#classes['color_scheme']=classes['class_name'].apply(lambda x:[random.randint(0,255) for i in range(3)])
classes['color_scheme']=[[0,255,0],[255,0,0],[0,0,255],[255,255,0],[255,0,255],[0,255,255],
                        [0,128,0],[128,0,0],[0,0,128],[128,128,0],[128,0,128],[0,128,128],
                        [0,102,204],[153,204,0],np.nan]
print("Classes that can be predicted :",len(classes),"entries")
classes

In [ ]:
num = len(train_csv['rad_id'].unique())
print('Number of unique radiologists : ',num)

In [ ]:
def read_xray(path, voi_lut = True, fix_monochrome = True):
    dicom = pydicom.read_file(path)
    
    # VOI LUT is used to transform raw DICOM data to "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
               
    # depending on this value, x-ray is inverted:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
     
    # normalize the numpy array
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
        
    return data

In [ ]:
def getBoxes(img_id):
    boxes=pd.DataFrame(columns=['class_name','class_id','rad_id','bbox'])
    for row in train_csv[train_csv['image_id']==img_id].to_numpy():
        
        # drop the 'No Finding' classes
        if not np.isnan(row[4]):
            boxes=boxes.append({'class_name':row[1],'class_id':row[2],'rad_id':row[3],'bbox':row[4:]},ignore_index=True)

    return boxes
    
def getImage_id(path):
    return path[61:93]

def getcolor(name,isLabel):
    if isLabel:
        out=list(classes[classes['class_name']==name]['color_scheme'])[0]
    else:
        out=list(classes[classes['class_name']== name.split(':')[0][:-1]]['color_scheme'])[0]
    return out
    
def drawBoxes(path,reduced=True,opacity=0.1,boxes=None):
    font=cv2.FONT_HERSHEY_SIMPLEX;
    isLabel = False
    
    # convert ndarray to grayscale with 3 channels
    img=cv2.cvtColor(read_xray(path),cv2.COLOR_GRAY2RGB)
    if boxes is None:
        isLabel = True
        if reduced:
            boxes=reduce_Boxes(getImage_id(path))
        else:
            boxes=getBoxes(getImage_id(path))
    
    # add annotations and bounding boxes
    for row in boxes.to_numpy():
        col = getcolor(row[0],isLabel)
        rect = np.uint8(np.ones((int(row[3][3]-row[3][1]), int(row[3][2]-row[3][0]), 3))*col)
        img=cv2.rectangle(img,(int(row[3][0]), int(row[3][1])),(int(row[3][2]), int(row[3][3])),col, 5)
        sub_combo = cv2.addWeighted(img[int(row[3][1]):int(row[3][3]),int(row[3][0]):int(row[3][2]),:], 1-opacity, rect, opacity, 1.0)    
        img[int(row[3][1]):int(row[3][3]),int(row[3][0]):int(row[3][2]),:] = sub_combo
        img=cv2.putText(img,row[0],(int(row[3][0]), int(row[3][1])-12),font,2,col,4)
    return img
        

In [ ]:
def plot_imgs(imgs,size=7,cols=4,reduced=False,boxes=None):
    rows=len(imgs)//cols + 1
    fig=plt.figure(figsize=(rows*size,cols*size))
    for i,path in enumerate(imgs):
        if boxes is None:
            img=drawBoxes(path,reduced)
        else:
            img=drawBoxes(path,reduced,boxes=boxes[i])
        fig.add_subplot(rows,cols, i+1)
        plt.imshow(img)
    plt.show()
    
def plot_img(path,reduced=False,figsize=(20,15),boxes=None):
    img=drawBoxes(path,reduced,boxes=boxes)
    fig=plt.figure(figsize=figsize)
    fig.add_subplot(1,1,1)
    plt.imshow(img)

In [ ]:
# output some randomly selected radiographs
plot_imgs(random.choices(Train_Dir_Files,k=8))

In [ ]:
#image_ids['count']=image_ids['Image_ids'].apply(lambda x: len(getBoxes(x)))

In [ ]:
most_anno={'Image_ids':'03e6ecfa6f6fb33dfeac6ca4f9b459c9','count':57}
least_anno={'Image_ids':'b7b84cc718930bb70aa2fdcb53cfb98e','count':3}

In [ ]:
#anno=image_ids.sort_values(['count'],ascending=False).reset_index(drop=True)
#most_anno=anno.iloc[0]
print("The image/radiograph with most annotations ("+str(most_anno['count'])+") : ")
plot_img(os.path.join(Train_Dir,str(most_anno['Image_ids'])+'.dicom'))

In [ ]:
#least_anno=anno[anno['count']!=0].sort_values(['count']).iloc[0]
print("The image/radiograph with least annotations ("+str(least_anno['count'])+") : ")
plot_img(os.path.join(Train_Dir,str(least_anno['Image_ids'])+'.dicom'))

In [ ]:
# plot the distribution of annotations per image
#plt.figure(figsize=(12,8))
#plt.xlabel('Number of Annotations per Image')
#plt.ylabel('Number of Images')
#sns.histplot(image_ids,bins=60)

In [ ]:
def drawBoxes_rad(path,rad,reduced=False,opacity=0.1):
    font=cv2.FONT_HERSHEY_SIMPLEX;
    
    # convert ndarray to grayscale with 3 channels
    img=cv2.cvtColor(read_xray(path),cv2.COLOR_GRAY2RGB)
    if reduced:
        old_boxes=reduced_Boxes(getImage_id(path))
    else:
        old_boxes=getBoxes(getImage_id(path))
    boxes=old_boxes[old_boxes['rad_id'] == rad]
    boxes.reset_index(drop=True,inplace=True)
    
    # add annotations and bounding boxes
    for row in boxes.to_numpy():
        rect = np.uint8(np.ones((int(row[3][3]-row[3][1]), int(row[3][2]-row[3][0]), 3))*getcolor(row[0],True))
        img=cv2.rectangle(img,
                (int(row[3][0]), int(row[3][1])),
                (int(row[3][2]), int(row[3][3])),
                getcolor(row[0],True), 5)
        sub_combo = cv2.addWeighted(img[int(row[3][1]):int(row[3][3]),int(row[3][0]):int(row[3][2]),:], 1-opacity, rect, opacity, 1.0)    
        img[int(row[3][1]):int(row[3][3]),int(row[3][0]):int(row[3][2]),:] = sub_combo
        img=cv2.putText(img,row[0],
                (int(row[3][0]), int(row[3][1])-12),
                 font,2,getcolor(row[0],True),4)
    return img

def plot_img_rad(path,size=30,cols=4,reduced=False):
    rads=list(train_csv[train_csv['image_id']==getImage_id(path)]['rad_id'].unique())
    rows=len(rads)//cols + 1
    fig=plt.figure(figsize=(rows*size,cols*size))
    for i,rad in enumerate(rads):
        img=drawBoxes_rad(path,rad,reduced)
        fig.add_subplot(rows,cols, i+1)
        plt.imshow(img)
    plt.show()
   

In [ ]:
 plot_img_rad(os.path.join(Train_Dir,str(most_anno['Image_ids'])+'.dicom'))

In [ ]:
boxes=getBoxes(most_anno['Image_ids'])
boxes[boxes['rad_id']=='R9']

In [ ]:
def calc_iou(b1,b2):
    x_left=max(b1[0],b2[0])
    x_right=min(b1[2],b2[2])
    y_top=max(b1[1],b2[1])
    y_bottom=min(b1[3],b2[3]) 
    if x_right < x_left or y_bottom < y_top:
        return 0
    else:
        intersec = max(0,x_right-x_left+1)*max(0,y_bottom-y_top+1)
        union = ((b1[2]-b1[0]+1)*(b1[3]-b1[1]+1)) + ((b2[2]-b2[0]+1)*(b2[3]-b2[1]+1)) - intersec
        return intersec/union

def getInnerBox(bboxes):
    xmin = max([box[0] for box in bboxes])
    ymin = max([box[1] for box in bboxes])
    xmax = min([box[2] for box in bboxes])
    ymax = min([box[3] for box in bboxes])
    if (xmax-xmin < 10) or (ymax-ymin < 10) :
        return None
    else:
        return [xmin, ymin, xmax, ymax]
            
def reduce_Boxes(img_id):
    boxes=getBoxes(img_id)
    new_boxes=pd.DataFrame(columns=['class_name','class_id','rad_id','bbox'])
    for row in boxes.to_numpy():
        rad=row[2]
        class_id=row[1]
        class_name=row[0]
        bbox=row[3]
        invalid_boxes=[bbox,]
        for row in boxes.to_numpy():
            if row[1] == class_id:
                other_box=row[3]
                if other_box.all() != bbox.all():
                    iou=calc_iou(bbox,other_box)
                    if iou > 0:
                        invalid_boxes.append(other_box)
        if (len(invalid_boxes)) >= 1:
            inner_box=getInnerBox(invalid_boxes)
            if inner_box and inner_box not in list(new_boxes['bbox']):
                new_boxes=new_boxes.append({'class_name':class_name,'class_id':class_id,
                                            'rad_id':rad,'bbox':inner_box},ignore_index=True)
    return new_boxes

In [ ]:
getBoxes(most_anno['Image_ids'])

In [ ]:
reduce_Boxes(most_anno['Image_ids'])

In [ ]:
plot_img(os.path.join(Train_Dir,str(most_anno['Image_ids'])+'.dicom'))

In [ ]:
plot_img(os.path.join(Train_Dir,str(most_anno['Image_ids'])+'.dicom'),True)

In [ ]:
not_a real_function()

In [ ]:
yolo_predict = pd.read_csv('../input/yolov5-sub/submission_yoloV5 (1).csv')
yolo_predict['PredictionString'] = yolo_predict['PredictionString'].apply(lambda x : list(map(float,x.split(' '))))
yolo_predict

In [ ]:
def getTestBoxes_yolo(img_id,conf=0.005):
    boxes=pd.DataFrame(columns=['label','class_id','conf','bbox'])
    predictions = list(yolo_predict[yolo_predict['image_id']==img_id]['PredictionString'])[0]
    for i in range(0,len(predictions),6):
        if predictions[i+1] >= conf:
            boxes=boxes.append({'label': (str(classes.iloc[int(predictions[i])]['class_name']) + " : "+str(predictions[i+1])),
                                'class_id':int(predictions[i]),
                                'conf': str(predictions[i+1]),
                                'bbox':predictions[i+2:i+6]},ignore_index=True)
    return boxes

In [ ]:
img_id = '0a08191a658edb1327e7282045ec71cf'
getTestBoxes_yolo(img_id,0.008)

In [ ]:
plot_img(f'{Test_Dir}/{img_id}.dicom',boxes=getTestBoxes_yolo(img_id,0.01))

In [ ]:
rand = random.choices(yolo_predict['image_id'],k=8)
boxes = [getTestBoxes_yolo(r) for r in rand]
files = [f'{Test_Dir}/{r}.dicom' for r in rand]
plot_imgs(files,boxes=boxes)

In [ ]:
from fastai import *
from fastai.vision.all import *

from sklearn.utils import shuffle

#from object_detection_fastai.helper.object_detection_helper import *
#from object_detection_fastai.loss.RetinaNetFocalLoss import RetinaNetFocalLoss
#from object_detection_fastai.models.RetinaNet import RetinaNet
#from object_detection_fastai.callbacks.callbacks import BBLossMetrics, BBMetrics, PascalVOCMetric

In [ ]:
#!pip install object-detection-fastai

In [ ]:
def get_train_fastai(debug=True,validate=True,use_reduced=False):
    dataset={}
    data=train_resized
    if validate:
        data=train_resized[:13500]
    if debug:
        data=train_resized[:1000]
    if use_reduced:
        Boxes=reduce_Boxes
    else:
        Boxes=getBoxes
        
    img_id=data['image_id'][0]
    img=cv2.imread(f'{Train_Dir_Resized}/{img_id}.png')
    height=img.shape[0]
    width=img.shape[1]
    
    for data_row in data.to_numpy():
        file = f'{Train_Dir_Resized}/{data_row[0]}.png'
        h_ratio=height/data_row[1]
        w_ratio=width/data_row[2]
        anno=[]
        cat=[]
        for anno_row in Boxes(data_row[0]).to_numpy():
            anno_rec=[round(anno_row[3][0]*w_ratio,3),
                      round(anno_row[3][1]*h_ratio,3),
                      round(anno_row[3][2]*w_ratio,3),
                      round(anno_row[3][3]*h_ratio,3)]
            cat_rec = anno_row[0]
            anno.append(anno_rec)
            cat.append(cat_rec)
        dataset[file]=[anno,cat]
    
    return dataset

In [ ]:
train_resized = shuffle(train_resized)
train_resized.reset_index(drop=True,inplace=True)

test_train=get_train_fastai()

train_demo=train_resized[:1000]
train_demo['filename'] = train_demo['image_id'].apply(lambda x: f'{Train_Dir_Resized}/{x}.png')
train_demo['bbox'] = train_demo['image_id'].apply(lambda x: test_train[f'{Train_Dir_Resized}/{x}.png'][0])
train_demo['bboxlbl'] = train_demo['image_id'].apply(lambda x: test_train[f'{Train_Dir_Resized}/{x}.png'][1])
train_demo['is_valid'] =  train_demo['image_id'].apply(lambda x: True)

train_demo.head()

In [ ]:
def get_pascal_train(x):
    return (x.filename,test_train[x][0],test_train[x][1])

In [ ]:
#chest = DataBlock(
#                  blocks=(ImageBlock, BBoxBlock, BBoxLblBlock),
 #                 splitter = IndexSplitter(train_demo.index),
  #                get_x = ColReader(3),
  #                 get_y =[ColReader(4),ColReader(5)],
  #                item_tfms = Resize(256),
  #                batch_tfms = aug_transforms(),
  #                n_inp = 1
  #                )

In [ ]:
#dls = chest.dataloaders(train_demo,num_workers=3)

In [ ]:
def get_train_dicts(debug=False,validate=True,use_reduced=False):
    dataset=[]
    data=train_resized
    if validate:
        data=train_resized[:13500]
    if debug:
        data=train_resized[:1000]
    if use_reduced:
        Boxes=reduce_Boxes
    else:
        Boxes=getBoxes
        
    img_id=data['image_id'][0]
    img=cv2.imread(f'{Train_Dir_Resized}/{img_id}.png')
    height=img.shape[0]
    width=img.shape[1]
    
    for data_row in data.to_numpy():
        record={}
        record['image_id']=data_row[0]
        record['file_name']=f'{Train_Dir_Resized}/{data_row[0]}.png'
        record['height']=height
        record['width']=width
        h_ratio=height/data_row[1]
        w_ratio=width/data_row[2]
        anno=[]
        for anno_row in Boxes(data_row[0]).to_numpy():
            anno_rec={}
            anno_rec['bbox']=[round(anno_row[3][0]*w_ratio,3),
                              round(anno_row[3][1]*h_ratio,3),
                              round(anno_row[3][2]*w_ratio,3),
                              round(anno_row[3][3]*h_ratio,3)]
            anno_rec['bbox_mode']=BoxMode.XYXY_ABS
            anno_rec['category_id']=anno_row[1]
            anno.append(anno_rec)
        record['annotations']=anno
        dataset.append(record)
    
    return dataset

def get_valid_dicts(debug=False,validate=True,use_reduced=False):
    dataset=[]
    data=train_resized
    if validate:
        data=train_resized[13500:]
        data.reset_index(drop=True,inplace=True)
    if debug:
        data=train_resized[2000:2100]
        data.reset_index(drop=True,inplace=True)
    if use_reduced:
        Boxes=reduce_Boxes
    else:
        Boxes=getBoxes
        
    img_id=data['image_id'][0]
    img=cv2.imread(f'{Train_Dir_Resized}/{img_id}.png')
    height=img.shape[0]
    width=img.shape[1]
    
    for data_row in data.to_numpy():
        record={}
        record['image_id']=data_row[0]
        record['file_name']=f'{Train_Dir_Resized}/{data_row[0]}.png'
        record['height']=height
        record['width']=width
        h_ratio=height/data_row[1]
        w_ratio=width/data_row[2]
        anno=[]
        for anno_row in Boxes(data_row[0]).to_numpy():
            anno_rec={}
            anno_rec['bbox']=[round(anno_row[3][0]*w_ratio,3),
                              round(anno_row[3][1]*h_ratio,3),
                              round(anno_row[3][2]*w_ratio,3),
                              round(anno_row[3][3]*h_ratio,3)]
            anno_rec['bbox_mode']=BoxMode.XYXY_ABS
            anno_rec['category_id']=anno_row[1]
            anno.append(anno_rec)
        record['annotations']=anno
        dataset.append(record)
    
    return dataset

def get_test_dicts(debug=False):
    dataset=[]
    data=test_resized
    if debug:
        data=test_resized[:50]
        
    img_id=data['image_id'][0]
    img=cv2.imread(f'{Test_Dir_Resized}/{img_id}.png')
    height=img.shape[0]
    width=img.shape[1]
    
    for data_row in data.to_numpy():
        record={}
        record['image_id']=data_row[0]
        record['file_name']=f'{Test_Dir_Resized}/{data_row[0]}.png'
        record['height']=height
        record['width']=width
        dataset.append(record)
        
    return dataset

def visualize_traindata(dicts,size=13,cols=4):
    meta_data=MetadataCatalog.get('Chest_abnormalities')
    rows=len(dicts)//cols + 1
    fig=plt.figure(figsize=(rows*size,cols*size))
    for i,d in enumerate(dicts):
        img = cv2.imread(d['file_name'])
        visualizer = Visualizer(img[:,:,::-1], metadata=meta_data)
        vis = visualizer.draw_dataset_dict(d)
        fig.add_subplot(rows,cols, i+1)
        plt.imshow(vis.get_image()[:,:,::-1])
    plt.show()
    
def visualize_testdata(dicts,predictor:DefaultPredictor,size=14,cols=4):
    meta_data=MetadataCatalog.get('Chest_abnormalities_test')
    rows=len(dicts)//cols + 1
    fig=plt.figure(figsize=(rows*size,cols*size))
    for i,d in enumerate(dicts):
        img = cv2.imread(d['file_name'])
        outputs = predictor(img)
        v = Visualizer(img[:, :, ::-1],
                   metadata=meta_data, 
                   scale=0.8,
                   instance_mode=ColorMode.IMAGE   
        )
        v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
        fig.add_subplot(rows,cols, i+1)
        plt.imshow(v.get_image()[:,:,::-1])
    plt.show()

def format_predictions(fields,dim0,dim1,resized_height=256,resized_width=256):
    preds=[]
    pred_classes=fields['pred_classes'].cpu().numpy()
    pred_boxes=fields['pred_boxes'].tensor.cpu().numpy()
    scores=fields['scores'].cpu().numpy()
    h_ratio = dim0 / resized_height
    w_ratio = dim1 / resized_width
    pred_boxes[:, [0, 2]] *= w_ratio
    pred_boxes[:, [1, 3]] *= h_ratio
    for label,score,box in zip(pred_classes,scores,pred_boxes):
        xmin, ymin, xmax, ymax =box.astype(np.int64)
        preds.append(f'{label} {score} {xmin} {ymin} {xmax} {ymax}')
    return " ".join(preds)

def get_test_predictions(predictor:DefaultPredictor):
    dataset=[]
    for test_row in test_resized.to_numpy():
        img=cv2.imread(f'{Test_Dir_Resized}/{test_row[0]}.png')
        out=predictor(img)
        if len(out['instances'])==0:
            pred_string='14 1.0 0 0 1 1'
        else:
            pred_string=format_predictions(out['instances'].get_fields(),test_row[1],test_row[2])
        record={'image_id':test_row[0],'PredictionString':pred_string,}
        dataset.append(record)
    return dataset

In [ ]:
DatasetCatalog.register('Chest_abnormalities',lambda: get_train_dicts())
MetadataCatalog.get('Chest_abnormalities').thing_classes=list(classes['class_name'][:14])
#DatasetCatalog.remove('Chest_abnormalities')
#MetadataCatalog.remove('Chest_abnormalities')

In [ ]:
#dataset_dicts = get_train_dicts()
#visualize_traindata(random.sample(dataset_dicts, 4))

In [ ]:
meta_data=MetadataCatalog.get('Chest_abnormalities')
meta_data.thing_classes

In [ ]:
DatasetCatalog.register('Chest_abnormalities_valid',lambda: get_valid_dicts())
MetadataCatalog.get('Chest_abnormalities_valid').thing_classes=list(classes['class_name'][:14])
#dataset_dicts=get_valid_dicts()

In [ ]:
DatasetCatalog.register('Chest_abnormalities_test',lambda: get_test_dicts())
MetadataCatalog.get('Chest_abnormalities_test').thing_classes=list(classes['class_name'][:14])
#dataset_dicts=get_test_dicts()

In [ ]:
class AlbumentationMapper:
    
    def __init__(self,cfg,is_train=True):
        augs = cfg.aug_kwargs;
        aug_list = []
        if is_train:
            aug_list.extend(getattr(A,func)(**args) for func,args in augs.items())
        self.transform = A.Compose(aug_list, bbox_params=A.BboxParams(format = 'pascal_voc',label_fields=["category_ids"]))
        self.is_train=is_train
        
    def __call__(self,dataset_dict):
        dataset = deepcopy(dataset_dict)
        old_anno = dataset['annotations']
        old_boxes = np.array([obj['bbox'] for obj in old_anno] , dtype=np.float32)
        cats = np.arange(len(dataset['annotations']))
        
        img = detection_utils.read_image(dataset['file_name'], format='BGR')
        trans_img = self.transform(image=img ,bboxes=old_boxes ,category_ids=cats)
        img = trans_img['image']
        new_anno=[]
        for i,j in enumerate(trans_img['category_ids']):
            d = old_anno[j]
            d['bbox'] = trans_img["bboxes"][i]
            new_anno.append(d)
        dataset.pop('annotations',None)
       
        img_shape = img.shape[:2]
        dataset["image"] = torch.as_tensor(img.transpose(2, 0, 1).astype("float32"))
        instances = detection_utils.annotations_to_instances(new_anno, img_shape)
        dataset["instances"] = detection_utils.filter_empty_instances(instances)
        return dataset

In [ ]:
class CustomTrainer(DefaultTrainer):
    
    @classmethod
    def build_train_loader(cls,cfg,sampler=None):
        #return build_detection_train_loader(cfg, mapper=AlbumentationMapper(cfg, True) ,sampler=sampler)
        return build_detection_train_loader(cfg, mapper=None ,sampler=sampler)
    
    @classmethod
    def build_test_loader(cls, cfg, dataset_name):
        #return build_detection_test_loader(cfg, dataset_name, mapper=AlbumentationMapper(cfg, False))
        return build_detection_test_loader(cfg, dataset_name, mapper=None)
        
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            output_folder = os.path.join(cfg.OUTPUT_DIR,"inference")
        return COCOEvaluator(dataset_name, ('bbox',), True, output_folder)

In [ ]:
cfg=get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))

cfg.aug_kwargs = CfgNode({"HorizontalFlip": {"p": 0.4},
                  "ShiftScaleRotate": {"scale_limit": 0.20, "rotate_limit": 20, "p": 0.4}})

cfg.OUTPUT_DIR = f'results/'

cfg.DATASETS.TRAIN = ('Chest_abnormalities',)
cfg.DATASETS.TEST = ('Chest_abnormalities_valid',)
cfg.DATALOADER.NUM_WORKERS = 4

cfg.MODEL.DEVICE='cuda'
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_SIZE = 512
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(meta_data.thing_classes)

cfg.SOLVER.MAX_ITER = 50000
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025

cfg.TEST.EVAL_PERIOD = 10000

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

In [ ]:
cfg.aug_kwargs

In [ ]:
#not_a_real_function(not_a_real_value)

In [ ]:
#trainer=CustomTrainer(cfg)
#trainer.resume_or_load(resume=False)
#trainer.train()
#not_a_real_function(not_a_real_value)

In [ ]:
submission = pd.read_csv('../input/data-subs-30000/submission (2).csv')
submission.to_csv("submission.csv", index=False)
submission

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR,'model_final.pth')
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
predictor=DefaultPredictor(cfg)

In [ ]:
dataset_dicts = get_test_dicts()
img = cv2.imread(dataset_dicts[1]['file_name'])
outputs = predictor(img)
outputs

In [ ]:
results=get_test_predictions(predictor)
submission = pd.DataFrame(results, columns=['image_id', 'PredictionString'])
submission.to_csv("submission_0.5.csv", index=False)
submission

In [ ]:
#not_a_real_function(not_a_real_value)

In [ ]:
SIZE = 256

In [ ]:
def preprocess_yolo():
    dataset = train_csv
    dataset = dataset.merge(train_resized, on='image_id', how='left')
    dataset['x_min_resize'] = round(dataset['x_min']*SIZE/dataset['width'],0)
    dataset['x_max_resize'] = round(dataset['x_max']*SIZE/dataset['width'],0)
    dataset['y_min_resize'] = round(dataset['y_min']*SIZE/dataset['height'],0)
    dataset['y_max_resize'] = round(dataset['y_max']*SIZE/dataset['height'],0)
    dataset['x_center'] = round((dataset['x_max_resize']+dataset['x_min_resize'])/2,1)
    dataset['y_center'] = round((dataset['y_max_resize']+dataset['y_min_resize'])/2,1)
    dataset['width_resize'] = round(dataset['x_max_resize']-dataset['x_min_resize'],1)
    dataset['height_resize'] = round(dataset['y_max_resize']-dataset['y_min_resize'],1)
    dataset = dataset[['image_id','class_name','class_id','rad_id','x_center','y_center','width_resize','height_resize']]
    dataset.rename(columns = {'width_resize':'w','height_resize':'h'},inplace=True)
    return dataset

def getXYHWBoxes(allbox):
    boxes=[]
    for row in allbox.to_numpy():
        # drop the 'No Finding' classes
        if not np.isnan(row[4]):
            boxes.append(f'{row[2]} {row[4]/SIZE} {row[5]/SIZE} {row[6]/SIZE} {row[7]/SIZE}')
    return boxes

def create_files_yolo(train_file='./chest-yolo',debug=False):
    data=preprocess_yolo()
    data = shuffle(data)
    data.reset_index(drop=True,inplace=True)
    if debug:
        train = data['image_id'].unique()[:500]
        val = data['image_id'].unique()[500:600]
    else:
        train = data['image_id'].unique()[:13000]
        val = data['image_id'].unique()[13000:]

    os.makedirs(f'{train_file}/images/train', exist_ok=True)
    os.makedirs(f'{train_file}/labels/train', exist_ok=True)
    os.makedirs(f'{train_file}/images/val', exist_ok=True)
    os.makedirs(f'{train_file}/labels/val', exist_ok=True)
    
    for img in train:
        boxes = getXYHWBoxes(data[data['image_id']==img])
        with open(f'{train_file}/labels/train/{img}.txt', 'w+') as f:
            for box in boxes:
                f.write(box)
                f.write('\n')
        sh.copy(f'{Train_Dir_Resized}/{img}.png',
                f'{train_file}/images/train/{img}.png')
        
    for img in val:
        boxes = getXYHWBoxes(data[data['image_id']==img])
        with open(f'{train_file}/labels/val/{img}.txt', 'w+') as f:
            for box in boxes:
                f.write(box)
                f.write('\n')
        sh.copy(f'{Train_Dir_Resized}/{img}.png',
                f'{train_file}/images/val/{img}.png')

In [ ]:
data = preprocess_yolo()
data.head()

In [ ]:
create_files_yolo()

In [ ]:
!git clone  'https://github.com/ultralytics/yolov5.git'

In [ ]:
!ls -R

In [ ]:
!mv yolov5/* ./
!mv ./* ./

In [ ]:
!ls

In [ ]:
!pip install -qr requirements.txt

In [ ]:
!echo -e 'train: ./chest-yolo/images/train\nval: ./chest-yolo/images/val\n\nnc: 14\nnames: ['Aortic enlargement','Atelectasis','Calcification','Cardiomegaly','Consolidation','ILD','Infiltration','Lung Opacity','Nodule/Mass','Other lesion','Pleural effusion','Pleural thickening','Pneumothorax','Pulmonary fibrosis']' >> bcc.yaml
!cat 'bcc.yaml'

In [ ]:
!sed -i 's/nc: 80/nc: 14/g' ./models/yolov5x.yaml

In [ ]:
with open('./models/yolov5x.yaml') as f:
    dictionary = yaml.load(f)
    for key,value in dictionary.items():
        print(key+':'+str(value))

In [ ]:
!wandb disabled

In [ ]:
!python ./train.py --epochs 20 --batch-size 4 --cfg ./models/yolov5x.yaml \
--data ./bcc.yaml --img 256 --name Chest1 --device 0

In [ ]:
from IPython.display import FileLink
FileLink("./runs/train/Chest1/weights/best.pt")

In [ ]:
#./runs/train/Chest1/weights/best.pt

In [ ]:
!python ./detect.py --weights ./runs/train/Chest1/weights/best.pt \
--img-size 256 --conf-thres 0.005 --source ../input/vinbigdata-chest-xray-resized-png-256x256/test --iou-thres 0.40 \
--save-txt --save-conf

In [ ]:
RESULTS = './runs/detect/exp/labels'

def process_labels(path,shape):
    with open(path,'r') as f:
        read = f.read()
        detect_result = re.split(r'[\n]', read)[:-1]
        records = []
        for row in detect_result:
            record = list(map(float,re.split(r'[ ]', row)))
            x_cen,y_cen = record[1]*SIZE,record[2]*SIZE
            w,h = record[3]*SIZE,record[4]*SIZE
            h_ratio = shape[0] / SIZE
            w_ratio = shape[1] / SIZE
            record[0] = int(record[0])
            record[1] = round(record[5],3)
            record[2] = int((x_cen - 0.5*w) * w_ratio)
            record[3] = int((y_cen - 0.5*h) * h_ratio)
            record[4] = int((x_cen + 0.5*w) * w_ratio)
            record[5] = int((y_cen + 0.5*h) * h_ratio)
            records.append(f'{record[0]} {record[1]} {record[2]} {record[3]} {record[4]} {record[5]}')
        return ' '.join(records)
    
def read_labels():
    dataset = []
    for row in test_resized.to_numpy():
        path = os.path.join(f'{RESULTS}/{row[0]}.txt')
        if os.path.isfile(path):
            results = process_labels(path,row[1:])
        else:
            results = '14 1 0 0 1 1'
        record={'image_id':row[0],'PredictionString':results,}
        dataset.append(record)
    return dataset
        

In [ ]:
results=read_labels()
submission = pd.DataFrame(results, columns=['image_id', 'PredictionString'])
submission.to_csv("submission_yoloV5.csv", index=False)
submission

In [ ]:
submission.shape

In [ ]:
from IPython.display import FileLink
FileLink(os.path.join("submission.csv"))

In [ ]:
dataset_dicts = get_test_dicts()

#Threshold_score = 0.5
sample_data=random.sample(dataset_dicts, 4)
visualize_testdata(sample_data,predictor)    

In [ ]:
#Threshold_score = 0
visualize_testdata(sample_data,predictor)  

In [ ]:
#evaluator = COCOEvaluator('Chest_abnormalities_valid','bbox',False, output_dir=cfg.OUTPUT_DIR)
#val_loader = build_detection_test_loader(cfg, 'Chest_abnormalities_valid')

#inference_on_dataset(predictor.model, val_loader, evaluator)

In [ ]:
#evaluator = COCOEvaluator("Chest_abnormalities_test",'bbox',False, output_dir=cfg.OUTPUT_DIR)
#val_loader = build_detection_test_loader(cfg, "Chest_abnormalities_test")

#inference_on_dataset(predictor.model, val_loader, evaluator)

In [ ]:
evals = pd.read_json('./results/metrics.json',orient='records',lines=True)
evals.tail()

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(16,6)) 
fig.suptitle('Loss Curves')
ax[0].plot(evals['iteration'],evals['loss_box_reg'])
ax[1].plot(evals['iteration'],evals['loss_cls'])
ax[0].set_xlabel('No. of Iterations')
ax[1].set_xlabel('No. of Iterations')
ax[0].set_ylabel('Classification Loss')
ax[1].set_ylabel('Box Regression Loss')
ax[1].set_ylim([0,0.6])

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(16,6)) 
fig.suptitle('ROIS per image')
ax[0].plot(evals['iteration'],evals['roi_head/num_fg_samples'])
ax[1].plot(evals['iteration'],evals['roi_head/num_bg_samples'])
ax[0].set_xlabel('No. of Iterations')
ax[1].set_xlabel('No. of Iterations')
ax[0].set_ylabel('No. of foreground proposals')
ax[1].set_ylabel('No. of foreground proposals')

In [ ]:
fig, ax = plt.subplots()
mdf_bbox_class = evals.iloc[-1][[f"bbox/AP-{col}" for col in meta_data.thing_classes]]
mdf_bbox_class.plot(kind="bar", ax=ax)
ax.set_title("AP by class")